# Dementia Risk Prediction

## Imports & Setup

In [ ]:
# Core libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# ML utilities
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Evaluation
from sklearn.metrics import classification_report, roc_auc_score, RocCurveDisplay


## Load Dataset

In [ ]:
df = pd.read_csv("../data/raw/nacc_hackathon.csv")
df.head()

df.shape
df.info()


## Target Variable

In [ ]:
TARGET = "Dementia"
df[TARGET].value_counts()


## Select a Valid Feature Set

In [ ]:
medical_features = [
    "MMSE",
    "CDR",
    "ADAS13",
    "Diagnosis",
    "ClinicalScore",
    "BrainVolume"
]

In [ ]:
non_medical_features = [
    col for col in df.columns
    if col not in medical_features + [TARGET]
]


In [ ]:
len(non_medical_features)


## Exploratory Data Analysis (EDA)

### Target distribution

In [ ]:
sns.countplot(x=TARGET, data=df)
plt.title("Dementia vs Non-Dementia Distribution")
plt.show()


### Missing values

In [ ]:
df[non_medical_features].isnull().mean().sort_values(ascending=False).head(10)


### Numerical distributions

In [ ]:
df[non_medical_features].select_dtypes("number").hist(
    figsize=(15,10), bins=20
)
plt.show()


### Categorical analysis

In [ ]:
df[non_medical_features].select_dtypes("object").nunique()


## Feature Engineering & Preprocessing

In [ ]:
X = df[non_medical_features]
y = df[TARGET]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


### Preprocessing pipelines

In [ ]:
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns


In [ ]:
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])


In [ ]:
categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])


In [ ]:
preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, num_cols),
    ("cat", categorical_pipeline, cat_cols)
])


## Build Binary Classification Models

### Logistic Regression (baseline)

In [ ]:
log_model = Pipeline([
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter=1000))
])

log_model.fit(X_train, y_train)


In [ ]:
log_probs = log_model.predict_proba(X_test)[:,1]
print("ROC-AUC:", roc_auc_score(y_test, log_probs))


### Random Forest

In [ ]:
rf_model = Pipeline([
    ("preprocess", preprocessor),
    ("model", RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        random_state=42
    ))
])

rf_model.fit(X_train, y_train)
